# COGS 108 - Final Project 

# Overview

Question: Does location and industry contribute to the overall success (defined by the total funding and market value) of a startup more than other factors?  
Hypothesis: The location and industry of startups are the main contributors to the overall success of a startup compared to other factors. We believe so because location is often correlated with different amount of business opportunities, and industry often dictates the profit margins and market.

# Names

- Hansae Lee
- Eun Hye (Grace) Oh
- Iris Nayki
- Matthew Davis
- Tianxing Liu
- Yuting Xu

#### Group Members IDs

- A14062618
- A12960475
- A14520625
- A13050499
- A14800461
- A13361405

# Research Question

*Does location and industry contribute to the overall success (defined by the total funding and market value) of a startup more than other factors?*

## Background and Prior Work

*Fill in your background and prior work here* 

References (include links):
- 1)
- 2)

# Hypothesis


*Hypothesis: The location and industry of startups are the main contributors to the overall success of a startup compared to other factors. We believe so because location is often correlated with different amount of business opportunities, and industry often dictates the profit margins and market.*

# Dataset(s)

*In order to answer our research question, we found Crunchbase (https://www.crunchbase.com), a platform for finding company data. They have datasets for companies and investors, built to track startups and thier finances. Crunchbases data is private and requires a subscription to access. We contacted Crunchbase as a team to ask for access to thier datasets for the purposes of this project. They kindly agreed to help our project and granted us an API key to download thier data set free of charge.* 

- Dataset Name: Crunchbase Daily Export
- Link to the dataset: 
    - https://github.com/mdavis917/COGS108_Repo
    - https://data.crunchbase.com/docs
    - https://www.crunchbase.com/
- Number of observations:

1-2 sentences describing each dataset. 

If you plan to use multiple datasets, add 1-2 sentences about how you plan to combine these datasets.

# Setup

In [1]:
import pandas as pd
import numpy as np
import operator
df = pd.read_csv('crunchbase_export.csv' , encoding='latin-1', low_memory = False)
df.shape

(235093, 23)

# Data Cleaning

Describe your data cleaning steps here.

In [2]:
df.drop(['company_name','region','domain','short_description', 'category_list', ' employee_count ','email','phone','cb_url', 'twitter_url', 'facebook_url' , 'uuid'], axis=1, inplace=True)
df = df[df['country_code'] == 'USA']
df.drop(['country_code'], axis=1, inplace=True)
df

,state_code,city,status,category_group_list,funding_rounds,funding_total_usd,founded_on,first_funding_on,last_funding_on,closed_on
8,MA,Somerville,operating,health care,4,"22,917,460",2011-10-22,2013-03-27,2019-05-09,NaN
17,NY,Albany,operating,biotechnology|education|financial services|hea...,1,"2,000,000",1839-01-01,2019-05-08,2019-05-08,NaN
18,MA,Tewksbury,operating,hardware|mobile,11,"325,543,279",2011-01-01,2012-04-26,2019-05-08,NaN
19,NC,Durham,operating,government and military|information technology...,6,"55,095,970",2011-01-01,2012-03-01,2019-05-08,NaN
22,CA,Playa Vista,operating,apps|commerce and shopping|food and beverage|i...,9,"64,036,900",2010-01-01,2012-01-01,2019-05-08,NaN
23,NC,Boone,operating,NaN,1,"300,000",2016-01-01,2019-05-08,2019-05-08,NaN
24,CA,San Francisco,operating,apps|information technology|sales and marketin...,5,"100,000,000",2011-01-01,2013-05-01,2019-05-08,NaN
26,NC,Asheville,operating,NaN,1,"300,000",2016-01-01,2019-05-08,2019-05-08,NaN
29,NC,Chapel Hill,operating,NaN,1,"300,000",2013-01-01,2019-05-08,2019-05-08,NaN
32,NC,Cary,operating,NaN,1,"300,000",2016-01-01,2019-05-08,2019-05-08,NaN


# Data Analysis & Results

Include cells that describe the steps in your data analysis.

## Most Successful By Location

There are many different cities where startups are created in the United States. As such we would like to make the data more readable by limiting it to a subset of the largest [VALUE WE CHOOSE] cities
- We can either import a population data set (like the homework) or just sort by the amount of companies in each city
- For cities that are below the threshold we bundle them in a group of other [MAY OR MAY NOT BE VALUABLE]

In addition, there are multiple criteria for success that we would like to explore. 
- Total amount of funding is a valuble metric for success in terms of raising funds. However, a startup can raise tons of capital and still burn through it all. 
- Success can also be measured by ability to be aquired, or launch as an IPO vs being closed, however a company can be aquired for pennies compared to how much it costs to fund. 
- Lastly we could use the longevity of a company as a metric of success. However a company could last with negative profits for a long time before finally closing
    
- We will explore all of these different metrics to see how they change the overall story of how location impacts these factors.

### Sorting and Cleaning Data for Location

### Total Companies by City

### Average Capital Raised

### Ratio of Closed to (Aquired & IPO)

### Longevity between first operation and last operation

## Most Successful By Industry

There are many industires that these startups are involved in. Some very unique topics and some very broad that don't say alot about what the company does. As such we need to find some of the most popular industies and see how each company fits into these. For each company, the data shows a list of categories the company is involved in. To clean this data we will create a bucket of the most popular 20 industies where each company that reports that industry will be in it. (Note some companies can be in multiple buckets) This list of 20 industries will have some broad terms like apps removed and replaced with the next highest topic occurance. This choice of non-specificity is done manually. And we have a description as to why the choice was removed.

We will then analyze this data by the same criteria of success as done in location

### Sorting and Cleaning Data By Industry 

First we sort by all of the industries listed for every company. We drop any that don't have an industry reported, rather than mark as other, because they might still belong in one of these other lists.

In [3]:
df_ind = df.copy()
df_ind.dropna(subset=['category_group_list'], inplace=True)

print(df_ind.shape)

topic_dict = {}
for index, row in df_ind.iterrows():
    categories_str = row['category_group_list']
    categories_lst = categories_str.split('|')
    for cat in categories_lst:
        topic_dict[cat] = topic_dict.get(cat, 0) + 1
        
sorted_topic = sorted(topic_dict.items(), key=operator.itemgetter(1), reverse = True)
for tup in sorted_topic:
    print("%s : %d" %tup)

(106754, 10)
software : 32822
health care : 18030
internet services : 17155
information technology : 16900
hardware : 14677
science and engineering : 13528
media and entertainment : 13010
financial services : 12115
manufacturing : 9946
commerce and shopping : 9506
data and analytics : 9503
mobile : 9249
sales and marketing : 8948
biotechnology : 6927
consumer electronics : 6095
real estate : 5225
transportation : 5007
food and beverage : 4540
advertising : 4536
apps : 4521
energy : 4343
design : 4260
content and publishing : 4199
education : 4185
community and lifestyle : 3846
professional services : 3572
privacy and security : 3505
lending and investments : 3404
sustainability : 3252
artificial intelligence : 3243
video : 3078
sports : 2918
natural resources : 2864
consumer goods : 2555
travel and tourism : 2442
payments : 1761
gaming : 1752
messaging and telecommunications : 1748
clothing and apparel : 1663
music and audio : 1604
administrative services : 1322
platforms : 1310
events

Now some of these topics are rather broad. I.E. (Software, Hardware, Apps, Internet Services, Information Technology, and Science and Engineering) So we will ignore these categories. Otherwise for each category a company has listed we will create a new row with just the one category

### Total Companies by Industry

### Average Capital Raised

### Ratio of Closed to (Aquired & IPO)

### Longevity between first operation and last operation

## Most Successful By Industry and Location

# Ethics & Privacy

*Fill in your ethics & privacy discussion here*

# Conclusion & Discussion

*Fill in your discussion information here*